In [35]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [36]:
train = pd.read_csv('../../ml-data/cyberattack_detection/2_processed_data/A2_training_data_by_ip.csv', sep=',')

In [37]:
X_train = train.drop(['Normal','SrcIPAddress'],1)
Y_train = train[['Normal']]

In [38]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns, index=X_train.index)

In [39]:
model = LogisticRegression(random_state=0,solver="liblinear").fit(X_train, Y_train)

/usr/local/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [40]:
test = pd.read_csv('../../ml-data/cyberattack_detection/2_processed_data/A2_test_data_by_ip.csv', sep=',')

In [41]:
X_test = test.drop(['Normal','SrcIPAddress'],1)
Y_test = test[['Normal']]
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

In [42]:
X_test.head()

,Duration,TotalPackets,TotalBytes,SourceBytes,PacketsBySec,SourceBytesBySec,TotalBytesBySec
0,-0.316295,-0.495782,-0.441498,-0.344372,0.145225,0.031500,0.051513
1,-0.192048,-0.424145,-0.396999,-0.252420,-0.133105,-0.115139,-0.161132
2,-0.195973,-0.318326,-0.281034,-0.207040,-0.127781,-0.112556,-0.147264
3,-0.271280,-0.404672,-0.360095,-0.299398,-0.110547,-0.104069,-0.104108
4,-0.265825,-0.277643,-0.232725,-0.203892,-0.123968,-0.110590,-0.136724


In [43]:
model.score(X_test,Y_test)

0.9298245614035088

In [44]:
predict = pd.DataFrame(model.predict(X_test),columns=['Predicted'])
result = pd.concat([test[['SrcIPAddress','Normal']].copy(),predict], axis=1)

In [45]:
result.head(5)

,SrcIPAddress,Normal,Predicted
0,122.226.12.150,1.0,0.0
1,147.101.94.182,0.0,0.0
2,147.113.94.198,0.0,0.0
3,147.114.98.192,0.0,0.0
4,147.122.97.192,0.0,0.0


In [18]:
ip = X_test.iloc[[1]]
label = Y_test.iloc[[1]]

In [19]:
label.head()

,Normal
1,0.0


In [25]:
Xadv, Y = adversarial_samples(ip.values, label.values, model, epsilon=0.2)

In [46]:
print(Xadv)

[[ 0.00795246 -0.22414549 -0.19699865 -0.05242023  0.06689528  0.08486123
   0.03886841]]


In [47]:
X_test.iloc[[1]] = Xadv

In [48]:
X_test.head()

,Duration,TotalPackets,TotalBytes,SourceBytes,PacketsBySec,SourceBytesBySec,TotalBytesBySec
0,-0.316295,-0.495782,-0.441498,-0.344372,0.145225,0.031500,0.051513
1,0.007952,-0.224145,-0.196999,-0.052420,0.066895,0.084861,0.038868
2,-0.195973,-0.318326,-0.281034,-0.207040,-0.127781,-0.112556,-0.147264
3,-0.271280,-0.404672,-0.360095,-0.299398,-0.110547,-0.104069,-0.104108
4,-0.265825,-0.277643,-0.232725,-0.203892,-0.123968,-0.110590,-0.136724


In [49]:
model.score(X_test,Y_test)

0.9122807017543859

In [50]:
predict = pd.DataFrame(model.predict(X_test),columns=['Predicted'])
result = pd.concat([test[['SrcIPAddress','Normal']].copy(),predict], axis=1)

In [51]:
result.head()

,SrcIPAddress,Normal,Predicted
0,122.226.12.150,1.0,0.0
1,147.101.94.182,0.0,1.0
2,147.113.94.198,0.0,0.0
3,147.114.98.192,0.0,0.0
4,147.122.97.192,0.0,0.0


In [21]:
def adversarial_samples(X, Y, model, epsilon):
    dlt = model.predict(X).T - Y.reshape(X.shape[0], 1)
    prob = model.predict_proba(X)
    dec = model.decision_function(X)
    #direction = np.sign(np.matmul(dlt, dec))
    direction = 1
    return X + epsilon * direction, Y